In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))
from json import loads
from pandas import read_parquet, to_numeric
from numpy import nan
from numpy.dtypes import StringDType
from config.constants import CLEANSED_ROUTE, RAW_ROUTE, FLIPKART_ECOMMERCE_SAMPLE_RAW, FLIPKART_ECOMMERCE_SAMPLE_CLEANSED
from utils.helpers import save_on_data_lake, replace_null_values, verified_nan_values

In [2]:
flipkart_parquet = read_parquet(f"{RAW_ROUTE}\\{FLIPKART_ECOMMERCE_SAMPLE_RAW}")

print(f"Cantidad de columnas: {len(flipkart_parquet.columns)}")
print(f"Nombre de columnas: {flipkart_parquet.columns}")

# Se eliminan las columnas inncesarias

columns_removed = [
    "crawl_timestamp",
    "product_url",
    "pid",
    "image",
    "is_FK_Advantage_product",
    "description",
    "product_specifications",
]

new_flipkart_df = flipkart_parquet.drop(columns_removed, axis=1)

print(f"Cantidad de columnas: {len(new_flipkart_df.columns)}")
print(f"Columnas actualizadas: {new_flipkart_df.columns}")

Cantidad de columnas: 15
Nombre de columnas: Index(['uniq_id', 'crawl_timestamp', 'product_url', 'product_name',
       'product_category_tree', 'pid', 'retail_price', 'discounted_price',
       'image', 'is_FK_Advantage_product', 'description', 'product_rating',
       'overall_rating', 'brand', 'product_specifications'],
      dtype='object')
Cantidad de columnas: 8
Columnas actualizadas: Index(['uniq_id', 'product_name', 'product_category_tree', 'retail_price',
       'discounted_price', 'product_rating', 'overall_rating', 'brand'],
      dtype='object')


In [3]:
# Se renombran las columnas
new_flipkart_df = new_flipkart_df.rename(columns={
    'uniq_id': 's_uniq_id',
    'product_name': 's_product_name',
    'product_category_tree': 's_product_category',
    'retail_price': 'd_retail_price',
    'discounted_price': 'd_discounted_price',
    'product_rating': 'd_product_rating',
    'overall_rating': 'd_overall_rating',
    'brand': 's_brand',
})

# Tipado de columnas
new_flipkart_df['s_uniq_id'] = new_flipkart_df['s_uniq_id'].astype(StringDType())
new_flipkart_df['s_product_name'] = new_flipkart_df['s_product_name'].astype(StringDType())
new_flipkart_df['s_product_category'] = new_flipkart_df['s_product_category'].astype(StringDType())
new_flipkart_df['d_retail_price'] = to_numeric(new_flipkart_df['d_retail_price'], errors='coerce')
new_flipkart_df['d_discounted_price'] = to_numeric(new_flipkart_df['d_discounted_price'], errors='coerce')
new_flipkart_df['d_product_rating'] = to_numeric(new_flipkart_df['d_product_rating'], errors='coerce')
new_flipkart_df['d_overall_rating'] = to_numeric(new_flipkart_df['d_overall_rating'], errors='coerce')
new_flipkart_df['s_brand'] = new_flipkart_df['s_brand'].astype(StringDType())


In [ ]:
def clean_product_category(category_tree: str)-> str:
    
    if (category_tree is nan):
        return "Not Category"
    
    # El método load convierte un array en string a un array literal
    category_array = loads(category_tree)
    main_category = category_array[0].split('>>')
    return main_category[0].strip()

# Limpieza de columnas
# Se remplazan las columnas necesarias por un verdadero nulo como nan
replace_null_values(new_flipkart_df)

# Se quitan los registros que no tiene id o sea que son NaN. Además, se usa el inplace para modificar el dataframe original
new_flipkart_df.dropna(subset=['s_uniq_id'], inplace=True)

numeric_columns = ['d_retail_price', 'd_discounted_price', 'd_product_rating', 'd_overall_rating']

# A todas las columnas que tengan un NaN se les colocará un 0
for column in numeric_columns:
    new_flipkart_df[column] = new_flipkart_df[column].fillna(0)

# Se limpian la columnas s_product_category
new_flipkart_df['s_product_category'] = new_flipkart_df['s_product_category'].apply(lambda x: clean_product_category(x))
new_flipkart_df['s_brand'] = new_flipkart_df['s_brand'].fillna('Unknow')

# Se verifican que no existan datos nulos
verified_nan_values(new_flipkart_df)

Cantidad de valores NaN en s_uniq_id: 0
Cantidad de valores NaN en s_product_name: 0
Cantidad de valores NaN en s_product_category: 0
Cantidad de valores NaN en d_retail_price: 0
Cantidad de valores NaN en d_discounted_price: 0
Cantidad de valores NaN en d_product_rating: 0
Cantidad de valores NaN en d_overall_rating: 0
Cantidad de valores NaN en s_brand: 0


In [5]:
# Se guarda el resultado final
save_on_data_lake(CLEANSED_ROUTE, new_flipkart_df, FLIPKART_ECOMMERCE_SAMPLE_CLEANSED)
new_flipkart_df

,s_uniq_id,s_product_name,s_product_category,d_retail_price,d_discounted_price,d_product_rating,d_overall_rating,s_brand
0,c2d766ca982eca8304150849735ffef9,Alisha Solid Women's Cycling Shorts,Clothing,999.0,379.0,0.0,0.0,Alisha
1,7f7036a6d550aaa89d34c77bd39a5e48,FabHomeDecor Fabric Double Sofa Bed,Furniture,32157.0,22646.0,0.0,0.0,FabHomeDecor
2,f449ec65dcbc041b6ae5e6a32717d01b,AW Bellies,Footwear,999.0,499.0,0.0,0.0,AW
3,0973b37acd0c664e3de26e97e5571454,Alisha Solid Women's Cycling Shorts,Clothing,699.0,267.0,0.0,0.0,Alisha
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,Sicons All Purpose Arnica Dog Shampoo,Pet Supplies,220.0,210.0,0.0,0.0,Sicons
...,...,...,...,...,...,...,...,...
19995,7179d2f6c4ad50a17d014ca1d2815156,WallDesign Small Vinyl Sticker,Baby Care,1500.0,730.0,0.0,0.0,WallDesign
19996,71ac419198359d37b8fe5e3fffdfee09,Wallmantra Large Vinyl Stickers Sticker,Baby Care,1429.0,1143.0,0.0,0.0,Wallmantra
19997,93e9d343837400ce0d7980874ece471c,Elite Collection Medium Acrylic Sticker,Baby Care,1299.0,999.0,0.0,0.0,Elite Collection
19998,669e79b8fa5d9ae020841c0c97d5e935,Elite Collection Medium Acrylic Sticker,Baby Care,1499.0,1199.0,0.0,0.0,Elite Collection
